<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Pawpularity_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle --upgrade

import os

os.environ['KAGGLE_USERNAME'] = 'kjohhhhh'
os.environ['KAGGLE_KEY'] = '9317f96d3b60d9f8d013f5e394d69311'

!kaggle competitions download -c petfinder-pawpularity-score

  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 13.3MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 9.08MB/s]
  0% 0.00/10.3k [00:00<?, ?B/s]
100% 10.3k/10.3k [00:00<00:00, 3.31MB/s]
  0% 0.00/10.3k [00:00<?, ?B/s]
100% 10.3k/10.3k [00:00<00:00, 9.40MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 10.7MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 10.2MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 11.1MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 10.4MB/s]
  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 41.6MB/s]
  0% 0.00/26.9k [00:00<?, ?B/s]
100% 26.9k/26.9k [00:00<00:00, 25.9MB/s]
  0% 0.00/119k [00:00<?, ?B/s]
100% 119k/119k [00:00<00:00, 37.6MB/s]
  0% 0.00/47.6k [00:00<?, ?B/s]
100% 47.6k/47.6k [00:00<00:00, 48.1MB/s]
  0% 0.00/77.8k [00:00<?, ?B/s]
100% 77.8k/77.8k [00:00<00:00, 80.1MB/s]
  0% 0.00/29.2k [00:00<?, ?B/s]
100% 29.2k/29.2k [00:00<0

In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models
!pip install --upgrade wandb

In [3]:
!pip install colorama

In [4]:
import os
import gc
import cv2
import copy
import time
import random
from PIL import Image

# for data manipulation
import numpy as np
import pandas as pd

# pytorch import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# sklearn import
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensor

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

# for descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [5]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret('wandb_api')
    wandb.login(key=api_key)
    anony=None
except:
    anony='must'
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


# 데이터 불러오기

In [13]:
ROOT_DIR = "/content"
TRAIN_DIR = "/content/train"
TEST_DIR = "/content/test"

In [8]:
# Traning Configure
CONFIG = dict(
seed=42,
model_name='tf_efficientnet_b4_ns',
train_batch_size=16,
valid_batch_size=32,
img_size=512,
epochs=5,
learning_rate=1e-4,
scheduler='CosineAnnealingLR',
min_lr=1e-6,
T_max=20,
T_0=25,
warmup_epochs=0,
weight_decay=1e-6,
n_accumulate=1,
n_fold=5,
num_classes=1,
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
competitions='PetFinder',
_wandb_kernel='deb')

In [9]:
# Set Seed for Reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=False
    os.environ['PYTHONHASHSEED']=str(seed)

set_seed(CONFIG['seed'])

In [10]:
def get_train_file_path(id):
    return f'{TRAIN_DIR}/{id}.jpg'


df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['file_path'] = df['Id'].apply(get_train_file_path)

feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]

In [11]:
# visualize images
run = wandb.init(project='Pawpularity',
                config=CONFIG,
                job_type='Visualization',
                anonymous='must')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
preview_table = wandb.Table(columns=['Id', 'Image', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity'])
tmp_df = df.sample(1000, random_state=CONFIG['seed']).reset_index(drop=True)
for i in tqdm(range(len(tmp_df))):
    row = tmp_df.loc[i]
    img=Image.open(row.file_path)
    preview_table.add_data(row['Id'],
                          wandb.Image(img),
                          row['Subject Focus'],
                          row['Eyes'],
                          row['Face'],
                          row['Near'],
                          row['Action'],
                          row['Accessory'],
                          row['Group'],
                          row['Collage'],
                          row['Human'],
                          row['Occlusion'],
                          row['Info'],
                          row['Blur'],
                          row['Pawpularity'])
wandb.log({'Visualization':preview_table})
run.finish()

  0%|          | 0/1000 [00:00<?, ?it/s]


NotADirectoryError: ignored

In [ ]:
# Creat Folds
def create_folds(df, n_s=5, n_grp=None):
    df['kfold'] = -1
    if n_grp is None:
        skf = KFold(n_splits=n_s, random_state=CONFIG['seed'])
        target=df['Pawpularity']
    else:
        skf=StratifiedKFold(n_splits=n_s, shuffle=True, random_state=CONFIG['seed'])
        df['grp']=pd.cut(df['Pawpularity'], n_grp, labels=False)
        target = df.grp
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        df.loc[v, 'kfold'] = fold_no
    
    df = df.drop('grp', axis=1)
    
    return df

In [ ]:
df = create_folds(df, n_s=CONFIG['n_fold'], n_grp=14)
df.head()

In [ ]:
# Dataset Class

class PawpularityDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir=root_dir  # 먼저 데이터 불러오기
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['Pawpularity'].values
        self.meta = df[feature_cols].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        meta = self.meta[index, :]
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        return img, meta, target

In [ ]:
# Augmentations

data_transforms = {
    'train' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225],
                   max_pixel_value=255.0,
                   p=1.0),
        ToTensorV2()
    ], p=1.0),
    
    'valid' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225],
                   max_pixel_value=255.0,
                   p=1.0),
        ToTensorV2()
    ], p=1.0)
}

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(PawpularityModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.n_features + 12, CONFIG['num_classes'])
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, images, meta):
        features = self.model(images)  # features = (bs, embedding_size)
        features = self.dropout(features)  
        features = torch.cat([features, meta], dim=1)  # features = (bs, embedding_size + 12)
        output = self.fc(features)  # outputs  = (bs, num_classes)
        return output

    
model = PawpularityModel(CONFIG['model_name'])
model.to(CONFIG['device'])

# 모델 설명 ↓↓

In [ ]:
# Loss Function
def criterion(outputs, targets):
    return torch.sqrt(nn.MSELoss()(outputs.view(-1), targets.view(-1)))

In [ ]:
# Training Function

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler=amp.GradScaler()
    
    dataset_size=0
    running_losss=0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, meta, targets) in bar:
        images = images.to(device, dtype=torch.float)
        meta = meta.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(images, meta)
            loss = criterion(outputs, targets)
            loss = loss / CONFIG['n_accumulate']
            
        scaler.scale(loss).backward()
        
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            scaler.step(optimizer)
            scaler.update()
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
    
    gc.collect()
    
    return epoch_loss
            

In [ ]:
# Validation function

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size=0
    running_loss=0.00
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, meta, targets) in bar:
        images = images.to(device, dtype=torch.float)
        meta = meta.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images, meta)
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        PREDS.append(outputs.view(-1).cpu().detach().numpy())
        TARGETS.append(targets.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_rmse = mean_squared_error(TARGETS, PREDS, squared=False)
    
    gc.collect()
    
    return epoch_loss, val_rmse

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print('[INFO] USING GPU: {}\n'.format(torch.cuda.get_device_name()))
        
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_rmse = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1):
        gc.collect()
        
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, dataloader=train_loader, device=CONFIG['device'], epoch=epoch)
        val_epoch_loss, val_epoch_rmse = valid_one_epoch(model, valid_loader, device=CONFIG['device'], epoch=epoch)
        
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid RMSE'].append(val_epoch_rmse)
        
        # Log the metrics
        wandb.log({'Train Loss':train_epoch_loss})
        wandb.log({'Valid Loss':val_epoch_loss})
        wandb.log({'Valid RMSE':val_epoch_rmse})
        
        # deep copy the model
        if val_epoch_rmse <= best_epoch_rmse:
            print(f"{c_}Validation Loos Improved ({best_epoch_rmse} --> {val_epoch_rmse})")
            best_epoch_rmse = val_epoch_rmse
            run.summary['Best RMSE'] = best_epoch_rmse
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = 'RMSE{:.4f}_epoch{:.04}.bin'.format(best_epoch_rmse, epoch)
            torch.save(model.state_dict(), PATH)
            wandb.save(PATH)
            print(f'Model Saved{sr_}')
            
        print()
    
    end=time.time()
    time_elapsed = end - start
    
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print('Best RMSE:{:.4f}'.format(best_epoch_rmse))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history


In [ ]:
def prepare_loader(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = PawpularityDataset(TRAIN_DIR, df_train, transforms=data_transforms['train'])
    valid_dataset = PawpularityDataset(TRAIN_DIR, df_valid, transforms=data_transforms['valid'])
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], num_workers=4, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], num_workers=4, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])
    
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=CONFIG['T_0'], eta_min=CONFIG['min_lr'])
    
    elif CONFIG['scheduler'] == None:
        return None
    
    return scheduler

In [ ]:
# create Dataloader
train_loader, valid_loader = prepare_loader(fold=0)

In [ ]:
# define optimizer and scheduler

optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

In [ ]:
run = wandb.init(project='Pawpularity',
                config=CONFIG,
                job_type='Train',
                anonymous='must')

In [ ]:
model, history = run_training(model, optimizer, scheduler, device=CONFIG['device'], num_epochs=CONFIG['epochs'])

In [ ]:
run.finish()